---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [36]:
import pandas as pd
import numpy as np

def blight_model():
    
    X_train = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory = False)
    addresses = pd.read_csv('addresses.csv')

    latlons = pd.read_csv ('latlons.csv')
    X_test = pd.read_csv('test.csv')
    ran_df = pd.merge(addresses, latlons, on = 'address')
    train = pd.merge(X_train, ran_df, on = 'ticket_id')
    test = pd.merge(X_test, ran_df, on = 'ticket_id')
    
    
    train.drop(['agency_name', 'inspector_name', 'violator_name', 'violation_street_number','violation_street_name', 'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code','address','state_fee', 'admin_fee',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 
        'violation_description','grafitti_status','balance_due','payment_amount','payment_date','payment_status','collection_status','compliance_detail'], axis = 1, inplace = True)
    from sklearn.preprocessing import LabelEncoder
    #'disposition', 'violation_code',
    
    
    L_rand = list(train.columns)
    L_rand.remove('compliance')
    test = test[L_rand]
    train = train.set_index('ticket_id')
    l1 = LabelEncoder()
    l1.fit(train['disposition'].append(test['disposition'], ignore_index=True))
    l1_label = l1.transform(train['disposition'])
    l1_label_test = l1.transform(test['disposition'])
    test['disposition'] = l1_label_test
    train['disposition'] = l1_label
    l2 = LabelEncoder()
    l2.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
    l2_label = l2.transform(train['violation_code'])
    l2_label_test = l2.transform(test['violation_code'])
    test['violation_code'] = l2_label_test
    train['violation_code'] = l2_label
    train.dropna(inplace = True)
    X_train_comp = train['compliance']
    train.drop(['compliance'], axis = 1, inplace = True)
    test = test.set_index('ticket_id')
    a1 = train['lat'].mean(skipna = True)
    a2 = train['lon'].mean(skipna = True)
    
    train['lat'] = train['lat'].fillna(a1)
    train['lon'] = train['lon'].fillna(a2)
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    from sklearn.ensemble import RandomForestRegressor
    
    
    model = RandomForestRegressor()
    grid_clf_auc = GridSearchCV(model,param_grid = {'n_estimators': [1, 40], 'max_depth': [8, 1]} , scoring = 'roc_auc')
    grid_clf_auc.fit(train, X_train_comp)

    #print('Grid best parameter (max. AUC): ', grid_clf_auc.best_params_)
    #print('Grid best score (AUC): ', grid_clf_auc.best_score_)
    for j in test.columns:
        u = test[j].mean(skipna = True)
        test[j] = test[j].fillna(u)
    return pd.DataFrame(grid_clf_auc.predict(test), test.index)

In [37]:
blight_model()

,0
ticket_id,
284932,0.127200
285362,0.013800
285361,0.057569
285338,0.073053
285346,0.076752
285345,0.073053
285347,0.079730
285342,0.951917
285530,0.023717
